In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime, timedelta

%matplotlib inline

class Config:
    def __init__(self):
        pass


def set_config(config_dict):
    config = Config()
    config.__dict__ = config_dict
    return config


In [2]:
config = set_config({
    'row_start': '2004-10-16',
    'row_end': '2004-12-14',
    "timestep": 2,
    "max_batch_size": 1,
    "layer_size": 1,
    "unit_size": 1,
    "dropout": 0,
    "learning_rate": 0.1,
    "max_epoch": 50,
    "time_col": 'tanggal',
    "num_predict": 5,
    "feature": ['rr'],
    # "feature": ['tn','tx','tavg','rh_avg','rr','ss','ff_x','ddd_x','ff_avg','ddd_car'],
    "prediction": 'rr'
})


def train_test_split(dataset, timestep=1):
    dataX, dataY = [], []
    for i in range(len(dataset) - timestep):
        dataX.append(dataset[i:(i + timestep)])
        dataY.append(dataset[i + timestep:i+timestep+1])
    return np.array(dataX), np.array(dataY)

def proyeksi_split(dataset, timestep=1):
    dataX = []
    for i in range(len(dataset) - timestep + 1):
        dataX.append(dataset[i:(i + timestep)])
    return np.array(dataX)

In [3]:
DATASETS = pd.read_csv('../Data/1985-2021.csv').replace(to_replace=[8888, 9999, 2555], value=np.nan)
DATASETS.interpolate(inplace=True)

for feature in DATASETS:
  if DATASETS[feature].dtypes == object and feature != config.time_col:
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=1)
    tokenizer.fit_on_texts(DATASETS[feature].astype(str))
    index_word = list(tokenizer.index_word.values())
    word_index = list(tokenizer.word_index.values())
    
    for index in index_word:
      DATASETS = DATASETS.replace(to_replace=str(index.upper()), value=tokenizer.word_index[index], regex=True)
      
DATASETS = DATASETS.loc[
  (DATASETS[config.time_col] >= config.row_start) & (DATASETS[config.time_col] <= config.row_end)
]

DATELIST = np.array([datetime.strptime(date, '%Y-%m-%d').date() for date in list(DATASETS[config.time_col])])

In [4]:
featuresets = DATASETS[config.feature]

scaller = MinMaxScaler()
featuresets_scaled = scaller.fit_transform(featuresets)

train_size = int(featuresets_scaled.shape[0] * 0.9)
trainset, testset = featuresets_scaled[0:train_size], featuresets_scaled[train_size:featuresets_scaled.shape[0]]
traindateset, testdateset = DATELIST[0:train_size], DATELIST[train_size:DATELIST.size]

X_train, y_train = train_test_split(trainset, timestep=config.timestep)
X_train = np.reshape(X_train, (X_train.shape[0], len(config.feature), X_train.shape[1]))
y_train = np.reshape(y_train, (y_train.shape[0], len(config.feature), y_train.shape[1]))

X_test, y_test = train_test_split(testset, timestep=config.timestep)
X_test = np.reshape(X_test, (X_test.shape[0], len(config.feature), X_test.shape[1]))
y_test = np.reshape(y_test, (y_test.shape[0], len(config.feature), y_test.shape[1]))

In [5]:
# y_pred_loss = list()
# y_true_loss = list()
# loss = list()


def mean_squared_error(y_true, y_pred):
    # y_pred_loss.append(y_pred)
    # y_true_loss.append(y_true)
    # loss.append(tf.keras.backend.mean(tf.keras.backend.square(y_pred - y_true)))
    return tf.keras.backend.mean(tf.keras.backend.square(y_pred - y_true))


def root_mean_squared_error(y_true, y_pred):
    return tf.keras.backend.sqrt(mean_squared_error(y_pred, y_true))

model = tf.keras.models.Sequential()

for i in range(0, config.layer_size):
    model.add(tf.keras.layers.LSTM(
        units=config.unit_size,
        return_sequences=True,
        batch_input_shape=(config.max_batch_size, len(config.feature), config.timestep),
        go_backwards=True,
        weights=[
            np.array([
                [0.5774, 0.5774, 0.5774, 0.5774],
                [0.5774, 0.5774, 0.5774, 0.5774]
            ]),
            np.array([
                [0.5774, 0.5774, 0.5774, 0.5774]
            ]),
            np.zeros([4])
        ]
    ))
else:
    # model.add(tf.keras.layers.Dense(units=1, activation='linear'))
    model.compile(
        optimizer=tf.keras.optimizers.SGD(learning_rate=config.learning_rate),
        loss=mean_squared_error,
        run_eagerly=True
    )
    model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (1, 1, 1)                 16        
                                                                 
Total params: 16
Trainable params: 16
Non-trainable params: 0
_________________________________________________________________


In [6]:
# from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard

batch_loss = list()

history = model.fit(
    X_train,
    y_train,
    shuffle=False,
    epochs=config.max_epoch,
    verbose=1,
    batch_size=config.max_batch_size,
    callbacks=[
        tf.keras.callbacks.LambdaCallback(
            on_epoch_begin=None,
            on_epoch_end=None,
            on_batch_begin=None,
            on_batch_end=lambda batch, logs=None: batch_loss.append(logs['loss']),
            on_train_begin=None,
            on_train_end=None,
        )
    ]
)

Epoch 1/50
52/52 [==============================] - 2s 34ms/step - loss: 0.0797
Epoch 2/50
52/52 [==============================] - 2s 33ms/step - loss: 0.0722
Epoch 3/50
52/52 [==============================] - 1s 28ms/step - loss: 0.0698
Epoch 4/50
52/52 [==============================] - 2s 32ms/step - loss: 0.0687
Epoch 5/50
52/52 [==============================] - 2s 34ms/step - loss: 0.0680
Epoch 6/50
52/52 [==============================] - 2s 35ms/step - loss: 0.0676
Epoch 7/50
52/52 [==============================] - 1s 29ms/step - loss: 0.0672
Epoch 8/50
52/52 [==============================] - 2s 39ms/step - loss: 0.0669
Epoch 9/50
52/52 [==============================] - 2s 31ms/step - loss: 0.0666
Epoch 10/50
52/52 [==============================] - 2s 33ms/step - loss: 0.0664
Epoch 11/50
52/52 [==============================] - 1s 25ms/step - loss: 0.0662
Epoch 12/50
52/52 [==============================] - 1s 26ms/step - loss: 0.0660
Epoch 13/50
52/52 [==================

In [7]:
# model.save('model.h5')

In [8]:
# model = tf.keras.models.load_model('model.h5')

In [9]:
# mse = model.evaluate(
#     X_test,
#     y_test,
#     verbose=1,
#     batch_size=config.max_batch_size,
# )

In [11]:
model.predict(X_test, verbose=1, batch_size=config.max_batch_size)

4/4 [==============================] - 0s 29ms/step


array([[[0.2002306 ]],

       [[0.18857674]],

       [[0.18828028]],

       [[0.18841618]]], dtype=float32)

In [ ]:
# testset_split = proyeksi_split(testset, timestep=config.timestep)
# testset_split_reshape = np.reshape(testset_split, (testset_split.shape[0], len(config.feature), testset_split.shape[1]))
# proyeksi = model.predict(testset_split_reshape, verbose=1, batch_size=config.max_batch_size)
# for i in range(0, proyeksi.shape[0]):
#     testset[i + config.timestep - 1] = np.reshape(proyeksi[i], (proyeksi[i].shape[1], len(config.feature)))

5/5 [==============================] - 1s 4ms/step


In [ ]:
# for i in range(0, config.num_predict):
#     testset_split = proyeksi_split(testset, timestep=config.timestep)
#     testset_split_reshape = np.reshape(testset_split, (testset_split.shape[0], len(config.feature), testset_split.shape[1]))
#     proyeksi = model.predict(testset_split_reshape, batch_size=config.max_batch_size)
#     testset = np.concatenate((testset, np.reshape(proyeksi[-1], (proyeksi[-1].shape[1], len(config.feature)))), axis=0)

9/9 [==============================] - 0s 3ms/step


In [ ]:
# testset = scaller.inverse_transform(testset)
# testdateset = np.concatenate((testdateset, pd.to_datetime(pd.date_range(DATELIST[-1] + timedelta(days=1), periods=config.num_predict, freq='1d')).date), axis=0)

# LABEL = np.concatenate((traindateset, testdateset), axis=0)

# PREDICTIONS = pd.DataFrame(testset, columns=[config.feature]).set_index(pd.Series(testdateset))
# PREDICTIONS.index = PREDICTIONS.index.to_series().apply(lambda x: datetime.strptime(x.strftime('%Y-%m-%d'), '%Y-%m-%d'))

# HISTORY = pd.DataFrame(np.array(DATASETS[config.feature]), columns=[config.feature]).set_index(pd.Series(DATELIST))
# HISTORY.index = HISTORY.index.to_series().apply(lambda x: datetime.strptime(x.strftime('%Y-%m-%d'), '%Y-%m-%d'))

# START_DATE_FOR_PLOTTING = (DATELIST[-1] - timedelta(days=max((config.num_predict) * 3, 90))).strftime("%Y-%m-%d")
# blank_index = (DATELIST[-(DATASETS.shape[0] - train_size + config.timestep - 1)]).strftime("%Y-%m-%d")

In [ ]:
# from pylab import rcParams

# rcParams['figure.figsize'] = 26, 10

# plt.plot(PREDICTIONS.loc[START_DATE_FOR_PLOTTING:].index, PREDICTIONS.loc[START_DATE_FOR_PLOTTING:][config.prediction], color='orange', label='Proyeksi')
# plt.plot(HISTORY.loc[START_DATE_FOR_PLOTTING:].index, HISTORY.loc[START_DATE_FOR_PLOTTING:][config.feature], label=config.feature)

# plt.axvline(x = min(PREDICTIONS.index), color='green', linewidth=2, linestyle='--')
# plt.grid(which='major', color='#cccccc', alpha=0.5)

# plt.title('Prediksi dan Histori Curah Hujan', family='Arial', fontsize=12)
# plt.xlabel('Timeline', family='Arial', fontsize=10)
# plt.ylabel('Tingkat Nilai', family='Arial', fontsize=10)
# plt.xticks(rotation=45, fontsize=8)
# plt.legend(shadow=True)
# plt.show()